In [4]:
import pandas as pd
sample = pd.read_excel('Data\sample.tsv.xlsx',header = None)
sample.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,776522983837954049,735449229028675584,2016-09-15 20:48:01,se lo dici tu... https://t.co/x7Qm1VHBKL,\N,\N,51c0e6b24c64e54e,\N,1.0, ,46.0027,8.96044,Twitter for iPhone,http://twitter.com/#!/download/iphone,plvtone filiae.,hazel_chb,146,110.0,28621.0,Earleen.


In [6]:
import pandas as pd
import sys
from imp import reload
sys.path.insert(0, "/media/diskD/EPFL/Fall 2016/ADA/Project/GMR_ADA_Project/EmotionAnalysis")
#from DataSchemaExtractionParsing import *
from DataPreProcessing import *
#from SentSemanticModule import *
#from SentTweetModule import *
#from SentSyntacticModule import *

In [4]:
w, h = 8, 2
matrix_sentence = [[0 for x in range(w)] for y in range(h)]
matrix_sentence

[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]

In [5]:
word_set = list(set(lexicon['Word']))
word_emotional_vectors = []
for word in word_set:
    word_emotional_vectors.append((word,list(lexicon[lexicon['Word']==word]['Score'])))
word_emotional_vectors_dict = dict(word_emotional_vectors)

NameError: name 'lexicon' is not defined

In [15]:
import pandas as pd
lexicon = pd.read_csv('NRCLexicon/NRC-Emotion-Lexicon-v0.92/NRC-emotion-lexicon.txt',engine='python', names=['col'])
lexicon['Word'] = lexicon['col'].str.split().apply(lambda x: x[0])
lexicon['EmotionCategory'] = lexicon['col'].str.split().apply(lambda x: x[1])
lexicon['Score'] = lexicon['col'].str.split().apply(lambda x: x[-1])
del(lexicon['EmotionCategory'])
del(lexicon['col'])

In [25]:
from tqdm import tqdm
distinct_words = list(set(lexicon['Word']))
word_vectors = []
for i in tqdm(range(0,len(distinct_words))):
    vector = list(lexicon[lexicon['Word']==distinct_words[i]]['Score'])
    word_vectors.append((distinct_words[i],vector))

100%|████████████████████████████████████| 14182/14182 [04:36<00:00, 50.22it/s]


TypeError: list indices must be integers or slices, not str

In [37]:
vector_dict = dict(word_vectors)
vector_dict['happy']
import numpy as np
#np.save('NRCLexicon/lexicon_dictionary.npy', vector_dict)
if 'aims' in vector_dict.keys():
    print ("yes")
list_ = [0 for i in range(0,10)]
list_
vector_dict['clap']

lexicon_dict = np.load('NRCLexicon/lexicon_dictionary.npy').item()
lexicon_dict['clap']

['0', '1', '0', '0', '1', '0', '1', '0', '0', '1']

# Data Extraction & Parsing

In [ ]:
sample_tweets = extract_tweets('Data/sample.tsv.xlsx')
# Keeping Meaningful columns
sample_tweets = sample_tweets[['createdAt', 'text', 'longitude', 'latitude', 'placeLongitude', 'placeLatitude', 'userLocation']]
sample_tweets.head()

In [ ]:
sample_tweets.shape

In [ ]:
set(sample_tweets['userLocation'])

In [ ]:
sample_tweets.columns

In [ ]:
#sample_tweets_cleaned = cleanTweets(sample_tweets)
sample_tweets_cleaned.head()

In [ ]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

In [ ]:
sample_tweets['text']

In [ ]:
#sample_tweets = sample_tweets.dropna()
#sample_tweets = sample_tweets.drop(sample_tweets.index[[2891]]).reindex()
#sample_tweets = sample_tweets.drop(sample_tweets.index[[2890]]).reindex()
#sample_tweets = sample_tweets.drop(sample_tweets.index[[2891]]).reindex() 
#sample_tweets = sample_tweets.drop(sample_tweets.index[[3916]]).reindex() 
#sample_tweets = sample_tweets.drop(sample_tweets.index[[4311]]).reindex()
#sample_tweets = sample_tweets.drop(sample_tweets.index[[7599]]).reindex()
sample_tweets.isnull().any()

In [ ]:
sample_tweets.iloc[864]

# Pre-Processing and Cleaning Tweets:

### Strategy:
1. Parsing and Character Encoding
2. Language Detection (Translation)
3. Remove Tweets for which geolocation (longitude  is not present or where geolocation does not correspond to Switzerland
4. Handling Entities/ Special categories: 
    4.1. Replacing @ instances with <username>
    4.2. Replacing urls with <url>
    4.3. Replacing Emoticons with their word meaning
    4.4. Replacing numbers/phone/fax with <number>
    4.5. Detecting place / city / country / any geolocation cues in any part of the tweet (#)
    4.6. Detecting time cues
5. Tokenization and replacing contractions
6. Part of Speech Tagging to recognize Affective words (Noun, Verbs, Adjectives, Adverb) 
7. Some NRE to replace basic entities like Proper Nouns with tag <proper_noun>
8. Stopwords and punctuation removal
9. (possibly spell-checking as well)
10. Lowering multiple occurences of a character in a word (words like soooooo => so)
11. Lemmatization and term normalization to get less variable versions of the same word. (possibly use thesaurus also) 
12. Remove less frequent words => word count + define a specific threshold

### 2. Language Detection & Translation

In [ ]:
from yandex_translate import YandexTranslate
translate = YandexTranslate('trnsl.1.1.20161128T210956Z.66d67a07332e46b7.66bee9bcc58c60428596f3e559b4a75a1944f839')

translated_list = []
for i in range(0,len(sample_tweets)):
    print i
    translated_list.append(translate.translate(sample_tweets.iloc[i]['text'], 'en')['text'])
sample_tweets['EnglishText'] = translated_list
sample_tweets.head()

### 3. Remove Tweets for which geolocation is not present or where geolocation does not correspond to Switzerland

### 4. Handling Special Entities / Categories:

#### 4.2. Replacing URLs in Tweets

### Tokenization of Tweets into words

In [ ]:
from nltk.tokenize import word_tokenize
 
word_tokenize(sample_tweets['text'])